In [9]:
# Install transformers from the API of hugging face
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
# Import the packages required
import numpy as np
import pandas as pd
from transformers import pipeline

In [11]:
df = pd.read_csv('/content/bbc_news.csv')
df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [12]:
df = df.loc[1:5000] # Taking a Portiion for simplicity sake
print(len(df))

5000


In [17]:
# function for the task
def apply_pipeline(df, text_col):
   
    # Load the pre-trained model for sentiment analysis and text classification

    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    # make a text classifier pipeline, note that the pipeline handles the tokenization of text for you
    text_classifier = pipeline("text-classification", model = model_name)
    # make a sentiment analyzer pipeline
    sentiment_analyzer = pipeline("sentiment-analysis", model = model_name)


    # Apply the text classification and sentiment classification pipeline to the 'description' column using apply() method
    # Alongside Extract the respective predicted labels from the result and add it into the dataframe
    df["classification"] = df[text_col].apply(lambda x: text_classifier(x)[0]["label"])
    df["sentiment_score"] = df[text_col].apply(lambda x: sentiment_analyzer(x)[0]["score"])
    df["sentiment_label"] = df[text_col].apply(lambda x: sentiment_analyzer(x)[0]["label"])

    # Return the resultant dataframe
    return df

# call the function
apply_pipeline(df, "description")
df.head()

,title,pubDate,guid,link,description,classification,sentiment_score,sentiment_label
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as...",NEGATIVE,0.965008,NEGATIVE
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...,NEGATIVE,0.503225,NEGATIVE
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...,POSITIVE,0.994798,POSITIVE
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...,NEGATIVE,0.972654,NEGATIVE
5,Ukraine war: PM to hold talks with world leade...,"Mon, 07 Mar 2022 08:33:29 GMT",https://www.bbc.co.uk/news/uk-60642926,https://www.bbc.co.uk/news/uk-60642926?at_medi...,Boris Johnson is to meet the Canadian and Dutc...,POSITIVE,0.996118,POSITIVE


In [15]:
df.to_csv("/content/result.csv", index=False)